In [1]:
import numpy as np

In [2]:
#Loads wavefunctions and potential from their respective files
def load_dataset():
    with open('ian_potentials.txt', 'r') as f:
        potential = np.loadtxt(f, delimiter = ', ')
        #print(np.shape(potential.T))

    with open('ian_wavefunctions.txt', 'r') as f:
        wavefunction = np.loadtxt(f, delimiter = ', ')
        #print(np.shape(wavefunction.T))
    
    return potential.T, wavefunction.T 
        
#X: potentials. X because it will the input 
#Y: wavefunctions. Y because it will be our labels

X, Y = load_dataset()


In [3]:
print(np.shape(X))

(1000, 50)


In [4]:
#Lets start with initialization. 
def initialize_parameters(n_x,n_y): # this is where we decide the NN architecture. Let's take n_x as an argument, since our first layer weights need to deal with that. Same idea with n_y
   
    parameters = {'W1':None,
                  'b1':None,
                  'W2':None,
                  'b2':None,
                  'W3':None,
                  'b3':None
                 }
    
    # randomly initializing parameters to be in Gaussian distribution, with mean 0 and variance 1
    parameters['W1'] = np.random.randn(5,n_x)*np.sqrt(2/(n_x)) # actually initializing w He initialization
    parameters['b1'] = np.random.randn(5,1)*np.sqrt(2/(n_x))
    parameters['W2'] = np.random.randn(6,5)*np.sqrt(2/(5))
    parameters['b2'] = np.random.randn(6,1)*np.sqrt(2/(5))
    parameters['W3'] = np.random.randn(n_y,6)*np.sqrt(2/(6))
    parameters['b3'] = np.random.randn(n_y,1)*np.sqrt(2/(6))
    return parameters

def initialize_gradients(parameters):
    gradients={
        'dW1':None,
        'db1':None,
        'dW2':None,
        'db2':None,
        'dW3':None,
        'db3':None
    }
    gradients['dW1'] = np.zeros(np.shape(parameters['W1']))
    gradients['db1'] = np.zeros(np.shape(parameters['b1']))
    gradients['dW2'] = np.zeros(np.shape(parameters['W2']))
    gradients['db2'] = np.zeros(np.shape(parameters['b2']))
    gradients['dW3'] = np.zeros(np.shape(parameters['W3']))
    gradients['db3'] = np.zeros(np.shape(parameters['b3']))
    return gradients

In [5]:
#Softmax allows us to denote probabilities because the total sum at the end will all add up to one
def softmax(ZL): # inputs the vector Z of the final layer
    denom = np.sum((np.exp(ZL-np.max(ZL))),axis=0) # subtract max(x) to shrink values & avoid exploding gradient
    return np.divide((np.exp(ZL-np.max(ZL))),denom)
def sigmoid(ZL): # NOTE: decided to go w relu on all just bc it's easier to do derivatives
    return 1/(1+np.exp(-ZL))
#We will probably just use relu 
def relu(ZL):
    return ZL.clip(min=0) # clips all negative elements of ZL, setting them to 0
def relu_back(AL): # computes derivative of relu fn, given the layer's activation
    mask = np.zeros(np.shape(AL))
    mask[AL>0]=1 # NOTE: technically, should be 1 or 0 based on ZL, not AL. But since AL = 0 iff ZL <= 0, this still works as shorthand
    mask[AL<=0]=0
#     print('AL is: ' +str(AL))
#     print('mask is: '+str(mask))
    return mask # return original value of AL if entry was nonnegative; 0 if negative

In [6]:
def forward_prop(X,Y,parameters,lambd=0): # given a set of parameters and our data, let's pass it forward and see what our guesses are.
    m = np.shape(X)[1]
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    #Z1 would be the activations of the entire layer. 'Vectorizing' saves alot of run time. 
    Z1 = np.dot(W1,X)+b1
    A1 = relu(Z1)
    
    Z2 = np.dot(W2,A1)+b2
    A2 = relu(Z2)

    Z3 = np.dot(W3,A2)+b3
    A3 = softmax(Z3)

    assert np.shape(A3) == np.shape(Y)

    cost = 1/m*(np.sum(np.multiply(Y,-np.log(A3))) + 0.5*lambd*(np.sum(W3**2)+np.sum(W2**2)+np.sum(W1**2)))# let's do the cross entropy loss

    cache = (Z1,Z2,Z3,X,A1,A2,lambd) # saving these babies for later
    
    return A3,cache,cost

In [7]:
def back_prop(A3,Y,cache, parameters,gradients,debug=False):    # Note: A3, Y, and cache can be of however many examples we happen to want to input at once
    (Z1,Z2,Z3,X,A1,A2,lambd) = cache
    m = np.shape(X)[1]
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    #We dont really care about dC/dz3 but this first derivative makes dC/dw3 much easier. dC/dz is given by the link. 
    dZ3 = A3-Y # someone calculated this here: https://deepnotes.io/softmax-crossentropy
    dW3 = 1/m*(np.dot(dZ3,A2.T) + lambd*np.sum(abs(W3)))
    db3 = 1/m*np.sum(dZ3,axis=1,keepdims=True) # so we don't get any rank 1 arrays

    if debug:
        print('Layer 3 Backprop:')
#         print('dA3 = '+str(np.average(dA3)))
        print('dZ3 = '+str(np.average(dZ3)))
        print('dW3 = '+str(np.average(dW3)))
        print('db3 = '+str(np.average(db3)))
        print('~'*30)
    
    #relu_back is the derivative of relu() 
    #It is easier to plug in dz/dw rather than dA/dw
    dA2 = np.dot(W3.T,dZ3) # basically running fwd prop but in reverse! using W3.T to get from dZ3 to A2, instead of the other way around! :D
    dZ2 = np.multiply(dA2,relu_back(A2)) # derivative of the relu fn
    dW2 = 1/m*(np.dot(dZ2,A1.T) + lambd*np.sum(abs(W2)))# include regulaization term
    db2 = 1/m*np.sum(dZ2,axis=1,keepdims=True)
    
    if debug:
        print('Layer 2 Backprop:')
#         print('dA2 = '+str(np.average(dA2)))
        print('dZ2 = '+str(np.average(dZ2)))
        print('dW2 = '+str(np.average(dW2)))
        print('db2 = '+str(np.average(db2)))
        print('~'*30)
    #We can basically copy the code for dA2
    dA1 = np.dot(W2.T,dZ2)
    dZ1 = np.multiply(dA1,relu_back(A1))
    dW1 = 1/m*(np.dot(dZ1,X.T) + lambd*np.sum(abs(W1)))
    db1 = 1/m*np.sum(dZ1,axis=1,keepdims=True)
    
    if debug:
        print('Layer 1 Backprop:')
#         print('dA1 = '+str(np.average(dA1)))
        print('dZ1 = '+str(np.average(dZ1)))
        print('dW1 = '+str(np.average(dW1)))
        print('db1 = '+str(np.average(db1)))
        print('~'*30)
    
    assert (np.shape(W1)==np.shape(dW1))
    assert (np.shape(b1)==np.shape(db1))
    assert (np.shape(W2)==np.shape(dW2))
    assert (np.shape(b2)==np.shape(db2))
    assert (np.shape(W3)==np.shape(dW3))
    assert (np.shape(b3)==np.shape(db3))
    
    
    gradients['dW1'] = dW1
    gradients['db1'] = db1
    gradients['dW2'] = dW2
    gradients['db2'] = db2
    gradients['dW3'] = dW3
    gradients['db3'] = db3
    
    
    return gradients

In [ ]:
def update_parameters(parameters,gradients,learning_rate=0.005,clip=False):
    if clip:
        parameters['W1'] = parameters['W1'] - learning_rate*np.clip(gradients['dW1'],-5,5)
        parameters['b1'] = parameters['b1'] - learning_rate*np.clip(gradients['db1'],-5,5)
        parameters['W2'] = parameters['W2'] - learning_rate*np.clip(gradients['dW2'],-5,5)
        parameters['b2'] = parameters['b2'] - learning_rate*np.clip(gradients['db2'],-5,5)
        parameters['W3'] = parameters['W3'] - learning_rate*np.clip(gradients['dW3'],-5,5)
        parameters['b3'] = parameters['b3'] - learning_rate*np.clip(gradients['db3'],-5,5)  
    else:
        parameters['W1'] = parameters['W1'] - learning_rate*gradients['dW1']
        parameters['b1'] = parameters['b1'] - learning_rate*gradients['db1']
        parameters['W2'] = parameters['W2'] - learning_rate*gradients['dW2']
        parameters['b2'] = parameters['b2'] - learning_rate*gradients['db2']
        parameters['W3'] = parameters['W3'] - learning_rate*gradients['dW3']
        parameters['b3'] = parameters['b3'] - learning_rate*gradients['db3']
    return parameters